# BeautifulSoup mini-homework

* 本次作業期望利用套件Beautiful及requests進行網頁抓取
* 目標網站為知名網站批踢踢實業坊之movie版
* 抓取最新之5頁內容
* 輸出csv檔，其中包含作者、看板、標題、時間、分數、內容
* 其中分數欄算法 : 推+1分、噓-1分


### Step1 : Import Package

In [1]:
import re
# Web scraping
from bs4 import BeautifulSoup
import requests
# Data manipulation
import numpy as np
import pandas as pd

### Step2 : Recommended Variable

In [2]:
# 從 movie 版首頁開始
url = "https://www.ptt.cc/bbs/movie/index.html"

# 準備要記錄的表格
df = pd.DataFrame(columns=["作者", "看板", "標題", "時間", "分數", "內容"])


### Step3 : Define a function

定義一個函數，可以使之回傳讀取完一篇文章所抓取到的6項變數資料
* 可能會使用到資料分析相關套件(如:pandas)
* 不需要的資料(如:公告、版規等)務必過濾掉


In [3]:
#so hard
def get_page_meta(url):
    result = {}
    html = requests.get(url)
    soup = BeautifulSoup(html.text, "html.parser")
    main = soup.find("div", id = "main-content")
    info = main.find("div", class_ = "article-metaline")
    for i in main.find_all("div", class_ = re.compile("article-metaline")):
        tag = i.find("span", class_ = ).text
        value = i.find("span", class_ = ).text
        result[tag] = value
    push = main.find_all("div", class_ = "push")
    for i in push:
        push_tag=[i.find("span", class_ = re.compile("push-tag")).text]
    for i in range(len(push_tag)):
        if push_tag[i] == "推 ": 
            push_tag[i] = 1
        elif push_tag[i] == "噓 ": 
            push_tag[i] = -1
        else: 
            push_tag[i] = 0
    result["分數"] = sum(push_tag)
    start = main.text.find("\n")
    end = main.text.find("※")
    content = main.text[start:end]
    for i in main.find_all(text = re.compile("※")):
        if i != '※ 文章網址: ':
            content += (i + "\n")
        else:
            content += (i + url +"\n")
    result["內容"] = content
    return result

### Step4 : Main Program

請仔細觀察該網站之原始碼規律，搭配上課提及的方法完成作業。
* 注意輸出csv檔之前須考慮編碼方式(可搭配 encoding="utf_8_sig")

In [4]:
for i in range(5):
    html_movie = requests.get(url)
    all_movie_soup = BeautifulSoup(html_movie.text, "html.parser")
    articles = all_movie_soup.find_all("div", class_ = "r-ent") 
    for article in articles:
        title = article.find("div", class_ = "title").text.strip()
        if (title.startswith("(本文已被刪除)") or title.startswith("[公告]")):
            continue
        new_url = "http://www.ptt.cc" + article.find("a").get("href")
        data = get_page_meta(new_url)
        df = df.append(data, ignore_index = True)
    btn_page = all_movie_soup.find("div", class_="btn-group btn-group-paging")
    url = "http://www.ptt.cc" + btn_page.find_all("a")[1].get("href")
df.to_csv('mini_homework_output.csv', index = False, encoding = "utf_8_sig")

UnboundLocalError: local variable 'push_tag' referenced before assignment